In [60]:
from tqdm import tqdm
import datetime as dt
import pymssql
import pandas as pd
import numpy as np
import warnings
import glob 
import os

warnings.filterwarnings('ignore')

In [67]:
conn = pymssql.connect(server='AX-SQL', database='Staging')
cursor = conn.cursor()

sql = f'''
SELECT [ШК], [Код склада], [Код товара], [Артикул поставщика], [Внутренний артикул], [ID Сайта], [Товарное направление],
[Товарная группа], [Товарный кластер], [Проба], [Размер изделия], [Тип изделия 1], [Тип изделия 2],
[Тип изделия 3], [Дизайн], [Бренд], [Гендерный признак], [Ценовая корзина], [Группа цен], [Остаток], [Цена Розн., за шт],
[Чистый вес], [Общий вес], [Цена изделия на бирке], [Скидка на изделии], [Запрет скидки], [Цена Закупки за 1 шт. (упр сс с НДС за единицу)], [Дата закупки]
from [Staging].[Reports].[Remainings_8h]
where [Товарное направление] in (N'ПДК', N'ДК')
'''

sql_query = pd.read_sql_query(sql, conn)
stock = pd.DataFrame(sql_query)

In [3]:
path = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Архив_отчеты\Продажи\По дням\\'
list_of_files = glob.glob(path + '*.xlsx') 
latest_file = max(list_of_files, key=os.path.getctime)

sklad = pd.read_excel(latest_file, sheet_name = 'Продажи по ассортименту', header = 2)

stock['Код склада'] = stock['Код склада'].astype('float64') 

sklad = sklad[['Unnamed: 0','Торговая марка', 'Юр лицо', 'Код склада', 'Дата открытия объекта', 'Код объекта', 'Город', 'Наименование объекта']]
sklad1 = sklad[['Код склада', 'Код объекта']].drop_duplicates().reset_index(drop=True)
sklad1 = sklad1[sklad1['Код объекта'].notna()]

sklad1['Код объекта'] = sklad1['Код объекта'].astype(str)
sklad1['8'] = sklad1['Код объекта'].apply(lambda x: x[0])
sklad1 = sklad1[sklad1['8'] != '8']

stock = stock.merge(sklad1, left_on='Код склада', right_on = 'Код склада', how='inner')

stock = stock.drop(columns=['8'])

In [4]:
stock['Дата закупки'] = pd.to_datetime(stock['Дата закупки'], dayfirst=True, format='mixed')

stock['Старая закупка (до 31.12.2022)'] = np.where(
    stock['Дата закупки'] <= pd.to_datetime('2022-12-31', dayfirst=True, format='mixed'), 1, 0
)
stock['Новая закупка (с 01.01.2023)'] = np.where(
    stock['Дата закупки'] >= pd.to_datetime('2023-01-01', dayfirst=True, format='mixed'), 1, 0
)

data = stock.groupby(by = 'ID Сайта', as_index=False)[['Старая закупка (до 31.12.2022)', 'Новая закупка (с 01.01.2023)']].sum()

data['Дата закупки'] = np.where(
    (data['Новая закупка (с 01.01.2023)'] > 0) & (data['Старая закупка (до 31.12.2022)'] > 0), 'Старые и новые',
    np.where(
        (data['Новая закупка (с 01.01.2023)'] > 0) & (data['Старая закупка (до 31.12.2022)'] == 0), 'Новые', 
        np.where(
            (data['Новая закупка (с 01.01.2023)'] == 0) & (data['Старая закупка (до 31.12.2022)'] > 0), 'Старые', 'Старые'
        )
    )
)

In [5]:
stock.drop(columns=['Старая закупка (до 31.12.2022)', 'Новая закупка (с 01.01.2023)', 'Дата закупки'], inplace=True)
data.drop(columns=['Старая закупка (до 31.12.2022)', 'Новая закупка (с 01.01.2023)'], inplace=True)
stock = stock.merge(data, how='outer', on='ID Сайта')

In [6]:
os.chdir(r'C:\\Users\Trenkin.Sergey\Desktop\sales')
extension = 'xlsx'
all_sales = [i for i in glob.glob('*.{}'.format(extension))]

sale = pd.DataFrame()
for name in tqdm(all_sales):
    sales = pd.read_excel(name)
    sale = pd.concat([sales, sale], ignore_index= True)

  0%|          | 0/9 [00:00<?, ?it/s]

100%|██████████| 9/9 [10:32<00:00, 70.32s/it]


In [7]:
objects_with_sales = stock['Код объекта'].astype(str).unique().tolist()

In [8]:
data = dt.datetime.isoweekday(dt.date.today())
date_now = dt.datetime.now().date().strftime('%Y-%m-%d')
last_monitoring = (dt.date.today() - dt.timedelta(days=90)).strftime('%Y-%m-%d')

sale = sale.query('КБК in @objects_with_sales')
sale = sale[sale['Дата'] >= last_monitoring]

In [9]:
images = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Артикул-фото 2024.06.11_.xlsx')
zapret = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\запрет Артикулы к размещению.xlsx')

In [10]:
stock.drop_duplicates(inplace = True)

In [11]:
compare_tov_id = stock[['Код товара', 'ID Сайта']]
compare_tov_id.columns = ['Тов', 'ID Сайта']

In [12]:
compare_tov_id.drop_duplicates(subset=['Тов'], inplace=True)

In [13]:
sale = sale.merge(compare_tov_id, how='left', on='Тов')
sale = sale.drop(columns=['Тов'])
sale = sale[['ID Сайта', 'Количество', 'Общая сумма']]
sale.dropna(inplace=True)
sale = sale.groupby(by = 'ID Сайта', as_index=False)[['Количество', 'Общая сумма']].sum()
sale.columns = ['ID Сайта', 'Количество продаж за последние 90 дней', 'Сумма продаж за последние 90 дней']

In [14]:
result = stock.drop_duplicates(subset=['ID Сайта'])
result = result.merge(sale, how='outer', on='ID Сайта')

In [15]:
ids = result['ID Сайта'].dropna().tolist()

In [16]:
images['article'] = images['article'].astype(str)
images = images.query('article in @ids')

In [17]:
images = images[['article', 'published', 'Ссылка на изображение']]
images.columns = ['ID Сайта', 'Опубликован', 'Ссылка на изображение']
result = result.merge(images, how='outer', on='ID Сайта')

In [18]:
result = result.drop_duplicates(subset=['ID Сайта'])
result = result[result['ID Сайта'] != '']

In [19]:
result = result.drop(columns=['Остаток'])

In [20]:
df = stock
df = df[df['ID Сайта'] != '']

In [21]:
sum_weight = df.groupby(by = 'ID Сайта', as_index = False)[['Чистый вес', 'Общий вес']].sum()
sum_weight = sum_weight.rename(columns = {'Чистый вес': 'Сумма по чистому весу', 'Общий вес': 'Сумма по общему весу'})

In [22]:
result = result.merge(sum_weight, how='inner', on='ID Сайта')

In [23]:
percentile_ = df.groupby(by = 'ID Сайта', as_index=False)['Цена Закупки за 1 шт. (упр сс с НДС за единицу)'].quantile(0.8)
percentile_ = percentile_.rename(columns={'Цена Закупки за 1 шт. (упр сс с НДС за единицу)': '80-й перцентиль по себестоимости'})

In [24]:
percentile_

,ID Сайта,80-й перцентиль по себестоимости
0,1000054,2294.970
1,1000091,4295.000
2,1000339,8124.630
3,1000450,218357.440
4,1000593,3372.440
...,...,...
28730,8380055,6384.892
28731,8380061,5253.316
28732,8380063,5590.150
28733,8380071,5440.430


In [25]:
result = result.merge(percentile_, how='inner', on='ID Сайта')

In [26]:
result

,ШК,Код склада,Код товара,Артикул поставщика,Внутренний артикул,ID Сайта,Товарное направление,Товарная группа,Товарный кластер,Проба,...,Цена Закупки за 1 шт. (упр сс с НДС за единицу),Код объекта,Дата закупки,Количество продаж за последние 90 дней,Сумма продаж за последние 90 дней,Опубликован,Ссылка на изображение,Сумма по чистому весу,Сумма по общему весу,80-й перцентиль по себестоимости
0,2078609049701,5705.0,ТОВ1309103,К115-8282Икорс,К115-8282Икорс_ДК,1455900,ДК,ДК КОЛЬЦА,ДК_2036,585,...,11450.59,4-07-015,Новые,36.0,969300.00,True,https://static2.585.cloud/media/products/e75ab...,535.836,592.59,12018.512
1,2078608278761,5705.0,ТОВ1309367,10-22-И000-80708,10-22-И000-80708_ДК,1426086,ДК,ДК СЕРЬГИ,ДК_2108,585,...,8481.67,4-07-015,Новые,66.0,1477127.00,True,https://static2.585.cloud/media/products/ebfcb...,513.444,548.30,8641.810
2,2078607997614,5705.0,ТОВ1309977,3158401ГИ,3158401ГИ_ДК,1698328,ДК,ДК ПОДВЕСКИ,ДК_2164,585,...,2423.94,4-07-015,Новые,85.0,537750.00,True,https://static2.585.cloud/media/products/a7dc2...,1249.630,1251.64,2537.690
3,2078608776919,5705.0,ТОВ1310018,21-33-1000-13435,21-33-1000-13435_ДК,1804661,ДК,ДК ПЕЧАТКИ,ДК_2003,585,...,14895.99,4-07-015,Новые,177.0,6221795.25,True,https://static2.585.cloud/media/products/a829e...,3540.097,3581.14,16159.184
4,2078608846028,5705.0,ТОВ1310095,ПСбр 1966,ПСбр 1966 _ДК,1041295,ДК,ДК СЕРЬГИ,ДК_2089,585,...,6361.58,4-07-015,Новые,309.0,3060297.96,True,https://static2.585.cloud/media/products/a5d14...,1728.098,1750.94,7032.260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28730,2078584570552,4745.0,ТОВ1245663,С122-4418ГР,С122-4418ГР_ПДК,1715277,ПДК,ПДК СЕРЬГИ,ДК_2545,585,...,9559.04,2-11-002,Старые,NaN,NaN,NaN,NaN,3.380,3.84,9559.040
28731,2078584550196,4745.0,ТОВ1245691,С124-6050ТР,С124-6050ТР_ПДК,1992170,ПДК,ПДК СЕРЬГИ,ДК_2554,585,...,10252.97,2-11-002,Старые,NaN,NaN,True,https://static2.585.cloud/media/products/8235a...,3.540,4.02,10252.970
28732,2078370261091,3963.0,ТОВ1053973,10100030076,10100030076_ДК,2221311,ДК,ДК ПОДВЕСКИ,ДК_2156,585,...,1648.72,2-09-105,Старые,NaN,NaN,NaN,NaN,0.710,0.71,1648.720
28733,2078584582814,3963.0,ТОВ1245126,К122-4591АМ,К122-4591АМ_ПДК,1185991,ПДК,ПДК КОЛЬЦА,ДК_2511,585,...,5264.31,2-09-105,Старые,NaN,NaN,NaN,NaN,1.850,2.10,5264.310


In [27]:
stock = stock.groupby(by = 'ID Сайта', as_index=False)['Остаток'].sum()
result = result.merge(stock, how='left', on='ID Сайта')

In [28]:
df['Чистый вес'] = df['Чистый вес'].astype(float)
clear_weight = df.groupby(by = 'ID Сайта', as_index=False)['Чистый вес'].agg({'min', 'median', 'max'})
clear_weight.rename(columns={'median': 'Чистый вес медиана', 'max': 'Чистый вес макс', 'min': 'Чистый вес мин'}, inplace=True)
clear_weight['Чистый вес размах, %'] = (clear_weight['Чистый вес макс'] / clear_weight['Чистый вес мин'] - 1) 
result = result.merge(clear_weight, how='left', on='ID Сайта')

In [29]:
price_birka = df.groupby(by = 'ID Сайта', as_index=False)['Цена Розн., за шт'].agg({'min', 'median', 'max'})
price_birka.rename(columns={'median': 'Цена изделия на бирке медиана', 'max': 'Цена изделия на бирке макс', 'min': 'Цена изделия на бирке мин'}, inplace=True)
result = result.merge(price_birka, how='left', on='ID Сайта')
result['Цена изделия на бирке размах, %'] = (result['Цена изделия на бирке макс'] / result['Цена изделия на бирке мин'] - 1) 

In [30]:
base_discount = df.groupby(by = 'ID Сайта', as_index=False)['Скидка на изделии'].agg({'min', 'max'})
base_discount.rename(columns={'max': 'Базовая скидка макс', 'min': 'Базовая скидка мин'}, inplace=True)
result = result.merge(base_discount, how='left', on='ID Сайта')

In [31]:
df['Цена со скидками'] = np.where(
    df['Запрет скидки'] == 'Да', df['Цена Розн., за шт'], df['Цена Розн., за шт'] * (1-df['Скидка на изделии'] / 100) * 0.8 * 0.6
)

In [32]:
price_skidki = df.groupby(by = 'ID Сайта', as_index=False)['Цена со скидками'].agg({'min', 'median', 'max'})
price_skidki.rename(columns={'median': 'Цена со скидками медиана', 'max': 'Цена со скидками макс', 'min': 'Цена со скидками мин'}, inplace=True)
price_skidki['Цена со скидками размах, %'] = (price_skidki['Цена со скидками макс'] / price_skidki['Цена со скидками мин'] - 1) 
result = result.merge(price_skidki, how='left', on='ID Сайта')

In [33]:
selfprice = df.groupby(by = 'ID Сайта', as_index=False)['Цена Закупки за 1 шт. (упр сс с НДС за единицу)'].agg({'min', 'median', 'max'})
selfprice.rename(columns={'median': 'Себестоимость медиана', 'max': 'Себестоимость макс', 'min': 'Себестоимость мин'}, inplace=True)
selfprice['Себестоимость размах, %'] = (selfprice['Себестоимость макс'] / selfprice['Себестоимость мин'] - 1) 
result = result.merge(selfprice, how='left', on='ID Сайта')

In [34]:
df['Наценка'] = df['Цена со скидками'] / df['Цена Закупки за 1 шт. (упр сс с НДС за единицу)'] - 1
markup = df.groupby(by = 'ID Сайта', as_index=False)['Наценка'].agg({'min', 'median', 'max'})
markup.rename(columns={'median': 'Наценка медиана', 'max': 'Наценка макс', 'min': 'Наценка мин'}, inplace=True)
result = result.merge(markup, how='left', on='ID Сайта')

In [35]:
zapret['Артикул сайта'] = zapret['Артикул сайта'].astype(str)
zapret = zapret['Артикул сайта'].tolist()

result['Запрет к размещению'] = np.nan
for i in result.index:
    if result['ID Сайта'][i] in zapret:
        result['Запрет к размещению'][i] = True
    else:
        result['Запрет к размещению'][i] = False

In [36]:
ecom = pd.read_excel(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\МЮР\ecom_stock_raw\Остатки 2024.06.28.xlsx')

In [37]:
path = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\discounts_zolotoy\\'
list_of_files = glob.glob(path + '\*.xlsx') 
latest_file = max(list_of_files, key=os.path.getctime)

discount_ecom = pd.read_excel(latest_file)

In [38]:
discount_ecom.columns = ['article', 'Опубликован', 'discount_value_promotions_1','discount_value_promotions_2', 'discount_value_promotions_3','discount_value_promotions_4']
ecom = ecom.merge(discount_ecom, how='left', on='article')

In [39]:
def discount_index(discount):
    try:
        return 1 - discount / 100
    except:
        return 1
    
ecom.discount_value_promotions_1 = ecom.discount_value_promotions_1.apply(discount_index)

ecom['Цена со скидками Ecom'] = np.where(
    (ecom['price'] == ecom['discount_price']) & (ecom['discount_value_promotions_1'] == 1), ecom['discount_price'] * ecom['discount_value_promotions_1'], 
    np.where(
        ecom['discount_value_promotions_1'] == 0.5, ecom['discount_price'] * ecom['discount_value_promotions_1'], ecom['discount_price'] * ecom['discount_value_promotions_1'] * 0.8
    )
)

In [40]:
ecom['article'] = ecom['article'].astype(str)
ecom = ecom[['article', 'Цена со скидками Ecom']]

In [41]:
ecom = ecom.query('article in @ids')
ecom.columns = ['ID Сайта', 'Цена со скидками Ecom']
result = result.merge(ecom, how='left', on='ID Сайта')

In [42]:
df['Сумма в себестоимости на остатках'] = df['Остаток'] * df['Цена Закупки за 1 шт. (упр сс с НДС за единицу)']

In [43]:
summa_ost = df.groupby(by ='ID Сайта', as_index=False)['Сумма в себестоимости на остатках'].sum()
result = result.merge(summa_ost, how='left', on='ID Сайта')

In [44]:
df['Цена за грамм'] = df['Цена со скидками'] / df['Чистый вес']
price_per_gramm = df.groupby(by = 'ID Сайта', as_index=False)['Цена за грамм'].agg({'min', 'median', 'max'})
price_per_gramm.rename(columns={'median': 'Цена за грамм медиана', 'max': 'Цена за грамм макс', 'min': 'Цена за грамм мин'}, inplace=True)
result = result.merge(price_per_gramm, how='left', on='ID Сайта')
result['Цена за грамм размах, %'] = (result['Цена за грамм макс'] / result['Цена за грамм мин'] - 1) 

In [45]:
zapret_sk = df.groupby(by = 'ID Сайта', as_index=False)['Запрет скидки'].count()

In [46]:
yes = df[df['Запрет скидки'] == 'Да']
no = df[df['Запрет скидки'] == 'Нет']

yes = yes.groupby(by = 'ID Сайта', as_index=False)['Запрет скидки'].count()
no = no.groupby(by = 'ID Сайта', as_index=False)['Запрет скидки'].count()

yes.columns = ['ID Сайта', 'Количество с запрещенной скидкой']
no.columns = ['ID Сайта', 'Количество с разрешенной скидкой']

yes = yes.merge(no, how='outer', on='ID Сайта')

result = result.merge(yes, how='left', on='ID Сайта')

In [47]:
result = result.drop_duplicates(subset=['ID Сайта'])

In [48]:
result['Размах по весу'] = np.where(
    result['Чистый вес размах, %'] >= 0.3, '3. Большой размах (>30%)',
    np.where(
        result['Чистый вес размах, %'] >= 0.1, '2. Средний размах (10-30%)', '1. Небольшой размах (<10%)'
    )
)

result['Размах по себестоимости'] = np.where(
    result['Себестоимость размах, %'] >= 0.5, '3. Большой размах (>50%)',
    np.where(
        result['Себестоимость размах, %'] >= 0.2, '2. Средний размах (20-50%)', '1. Небольшой размах (<10%)'
    )
)

result['Размах по цене со скидками'] = np.where(
    result['Цена со скидками размах, %'] >= 0.5, '3. Большой размах (>50%)',
    np.where(
        result['Цена со скидками размах, %'] >= 0.2, '2. Средний размах (20-50%)', '1. Небольшой размах (<10%)'
    )
)

result['Размах по цене за грамм'] = np.where(
    result['Цена за грамм размах, %'] >= 0.3, '3. Большой размах (>30%)',
    np.where(
        result['Цена за грамм размах, %'] >= 0.1, '2. Средний размах (10-30%)', '1. Небольшой размах (<10%)'
    )
)

result['Размах цена изделия на бирке'] = np.where(
    result['Цена изделия на бирке размах, %'] >= 0.5, '3. Большой размах (>50%)',
    np.where(
        result['Цена изделия на бирке размах, %'] >= 0.2, '2. Средний размах (20-50%)', '1. Небольшой размах (<20%)'
    )
)

In [49]:
result['Скидка разрешена'] = result['Количество с запрещенной скидкой'] + result['Количество с разрешенной скидкой']

result['Скидка разрешена'] = np.where(
    result['Скидка разрешена'].astype(str).str.contains('nan'), 'Да или нет', 'Да и нет'
)

In [50]:
result.columns


Index(['ШК', 'Код склада', 'Код товара', 'Артикул поставщика',
       'Внутренний артикул', 'ID Сайта', 'Товарное направление',
       'Товарная группа', 'Товарный кластер', 'Проба', 'Размер изделия',
       'Тип изделия 1', 'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
       'Гендерный признак', 'Ценовая корзина', 'Группа цен',
       'Цена Розн., за шт', 'Чистый вес', 'Общий вес', 'Цена изделия на бирке',
       'Скидка на изделии', 'Запрет скидки',
       'Цена Закупки за 1 шт. (упр сс с НДС за единицу)', 'Код объекта',
       'Дата закупки', 'Количество продаж за последние 90 дней',
       'Сумма продаж за последние 90 дней', 'Опубликован',
       'Ссылка на изображение', 'Сумма по чистому весу',
       'Сумма по общему весу', '80-й перцентиль по себестоимости', 'Остаток',
       'Чистый вес макс', 'Чистый вес медиана', 'Чистый вес мин',
       'Чистый вес размах, %', 'Цена изделия на бирке макс',
       'Цена изделия на бирке медиана', 'Цена изделия на бирке мин',
       '

In [51]:
result = result.drop(columns = ['ШК', 'Код склада', 'Код объекта', 'Чистый вес', 'Цена изделия на бирке', 'Размер изделия', 'Общий вес', 'Скидка на изделии', 'Запрет скидки', 'Цена Закупки за 1 шт. (упр сс с НДС за единицу)',
                                'Запрет скидки', 'Цена Розн., за шт', 'Код товара'])

In [52]:
tov = result.groupby(by = 'Товарная группа')['Остаток'].sum()
tov_sales = result.groupby(by = 'Товарная группа')['Количество продаж за последние 90 дней'].sum()

result['Доля % остатка ID отосительно остатка ТГ'] = np.nan
for i in result.index: 
    t = result['Товарная группа'][i]
    result['Доля % остатка ID отосительно остатка ТГ'][i] = result['Остаток'][i] / tov[t] 
    
    
result['Доля % относительно продаж ТГ за последние 90 дней'] = np.nan
for i in result.index: 
    t = result['Товарная группа'][i]
    result['Доля % относительно продаж ТГ за последние 90 дней'][i] = result['Количество продаж за последние 90 дней'][i] / tov_sales[t] 

In [54]:
result.to_excel(r'C:\Users\Trenkin.Sergey\Desktop\notebooks\result.xlsx', index=False)

In [ ]:
date_today = dt.datetime.today().strftime('%Y-%m-%d')

In [ ]:
result.to_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\Исследование цен\\Исследование цен в разрезе ID {date_today}.xlsx', index=False)
# for tn in result['Товарное направление'].unique().tolist():
#     result[result['Товарное направление'] == tn].\
#         to_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\Исследование цен\Исследование цен для {tn} в разрезе ID {date_today}.xlsx', index=False)